In [8]:
# This is the second of four code batches.
# Run this first - it will download and locally save a weather dataframe that will be left merged with 
# the Los Angeles Crime dataset.

In [9]:
# Source:
# https://open-meteo.com/en/docs/historical-weather-api?hourly=temperature_2m,rain&daily=temperature_2m_max&start_date=2010-01-01&end_date=2024-12-11&csv_coordinates=los+angeles%0A&temperature_unit=fahrenheit&wind_speed_unit=mph&precipitation_unit=inch&bounding_box=-90,-180,90,180&latitude=34.0522&longitude=-118.2437&timezone=America%2FLos_Angeles

# If you need to install:
# pip install openmeteo-requests
# pip install requests-cache retry-requests numpy pandas


In [10]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 34.0522,
	"longitude": -118.2437,
	"start_date": "2009-12-31",
	"end_date": "2025-12-31",
	"daily": "temperature_2m_max",
	"timezone": "America/Los_Angeles",
	"temperature_unit": "fahrenheit",
	"wind_speed_unit": "mph",
	"precipitation_unit": "inch",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["temperature_2m_max"] = daily_temperature_2m_max

daily_dataframe = pd.DataFrame(data = daily_data)
print("\nDaily data\n", daily_dataframe)

Coordinates: 34.05975341796875°N -118.23750305175781°E
Elevation: 91.0 m asl
Timezone: b'America/Los_Angeles'b'GMT-8'
Timezone difference to GMT+0: -28800s

Daily data
                           date  temperature_2m_max
0    2009-12-31 08:00:00+00:00           67.011803
1    2010-01-01 08:00:00+00:00           70.611801
2    2010-01-02 08:00:00+00:00           76.551804
3    2010-01-03 08:00:00+00:00           76.821800
4    2010-01-04 08:00:00+00:00           75.111801
...                        ...                 ...
5840 2025-12-27 08:00:00+00:00           60.080002
5841 2025-12-28 08:00:00+00:00           64.849998
5842 2025-12-29 08:00:00+00:00           69.440002
5843 2025-12-30 08:00:00+00:00           67.730003
5844 2025-12-31 08:00:00+00:00           64.940002

[5845 rows x 2 columns]


In [11]:
daily_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5845 entries, 0 to 5844
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   date                5845 non-null   datetime64[ns, UTC]
 1   temperature_2m_max  5845 non-null   float32            
dtypes: datetime64[ns, UTC](1), float32(1)
memory usage: 68.6 KB


In [12]:
daily_dataframe.to_csv('Los Angeles Temps 2010 to 2025.csv')